In [ ]:
%pip install -U transformers huggingface_hub accelerate datasets -q

In [ ]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

### Использование предварительно обученных трансформеров
_по приколу_

Существует множество инструментов, позволяющих получить доступ к предварительно обученным моделям-трансформерам, но наиболее мощным и удобным из них является [`huggingface/transformers`](https://github.com/huggingface/transformers). На этой неделе вы научитесь скачивать, применять и модифицировать предварительно обученные трансформеры для различных задач. Пряжем ремни, мы отправляемся в путь!

__Пайплайны:__ если вы хотите просто применить предварительно обученную модель, вы можете сделать это одной строкой кода с использованием pipeline. Huggingface/transformers предлагает ряд предварительно настроенных пайплайнов для маскированного языкового моделирования, классификации эмоций, ответов на вопросы и т.д. ([полный список здесь](https://huggingface.co/transformers/main_classes/pipelines.html))

Типичный пайплайн включает:
* предварительную обработку, например, токенизацию, сегментацию на подслова
* базовую модель, например, bert, настроенный для классификации
* постобработку результатов

Давайте посмотрим это на практике:

In [ ]:
import transformers

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="s-nlp/russian_toxicity_classifier")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

NameError: name 'classifier' is not defined

In [ ]:
print(pipe("собака очень крутая!!"))

[{'label': 'neutral', 'score': 0.9997190833091736}]


In [ ]:
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

for k, v in data.items():
  print(k)
  print(classifier(v), "\n")

arryn
[{'label': 'POSITIVE', 'score': 0.9997355341911316}] 

baratheon
[{'label': 'NEGATIVE', 'score': 0.6957710385322571}] 

stark
[{'label': 'POSITIVE', 'score': 0.9950939416885376}] 

tyrell
[{'label': 'POSITIVE', 'score': 0.9998761415481567}] 



Вы также можете получить доступ к базовой модели Masked Language Model, которая была обучена предсказыванию замаскированных слов. Вот как это делается:

In [ ]:
mlm_model = transformers.pipeline('fill-mask', model="cointegrated/rubert-tiny2")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"{MASK}"):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

P=0.13561 Берт вышел в 20 лет году
P=0.08163 Берт вышел в 20 м году
P=0.05174 Берт вышел в 20 августа году
P=0.04394 Берт вышел в 20 марта году
P=0.04168 Берт вышел в 20 февраля году


In [ ]:
# Ваша очередь: используйте bert, чтобы вспомнить, в каком году вышла Dota 2
mlm_model(f"Dota2 released in 20{MASK} year")

[{'score': 0.45413684844970703,
  'token': 1011,
  'token_str': '-',
  'sequence': 'dota2 released in 20 - year'},
 {'score': 0.19154523313045502,
  'token': 2944,
  'token_str': 'model',
  'sequence': 'dota2 released in 20 model year'},
 {'score': 0.06123613566160202,
  'token': 1009,
  'token_str': '+',
  'sequence': 'dota2 released in 20 + year'},
 {'score': 0.05442217364907265,
  'token': 2243,
  'token_str': '##k',
  'sequence': 'dota2 released in 20k year'},
 {'score': 0.02352585457265377,
  'token': 2086,
  'token_str': 'years',
  'sequence': 'dota2 released in 20 years year'}]

```

```

```

```


Huggingface предлагает сотни предварительно обученных моделей, специализирующихся на различных задачах. Вы можете быстро найти нужную модель, используя [этот список](https://huggingface.co/models).


In [ ]:
text = """
Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet
for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
“We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Задача: создать пайплайн для распознавания именованных сущностей, использовать название задачи 'ner' и найти подходящую модель в списке
ner_model = transformers.pipeline('ner', model='dslim/bert-base-NER')

named_entities = ner_model(text)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
named_entities

[{'entity': 'B-LOC',
  'score': 0.79910463,
  'index': 27,
  'word': 'Rose',
  'start': 112,
  'end': 116},
 {'entity': 'I-LOC',
  'score': 0.9511927,
  'index': 28,
  'word': '##tta',
  'start': 116,
  'end': 119},
 {'entity': 'B-ORG',
  'score': 0.998223,
  'index': 40,
  'word': 'Guardian',
  'start': 179,
  'end': 187},
 {'entity': 'B-PER',
  'score': 0.9997613,
  'index': 46,
  'word': 'Ian',
  'start': 206,
  'end': 209},
 {'entity': 'I-PER',
  'score': 0.99978715,
  'index': 47,
  'word': 'Sam',
  'start': 210,
  'end': 213},
 {'entity': 'I-PER',
  'score': 0.99964595,
  'index': 48,
  'word': '##ple',
  'start': 213,
  'end': 216},
 {'entity': 'B-PER',
  'score': 0.9997831,
  'index': 53,
  'word': 'Stuart',
  'start': 239,
  'end': 245},
 {'entity': 'I-PER',
  'score': 0.9997482,
  'index': 54,
  'word': 'Clark',
  'start': 246,
  'end': 251},
 {'entity': 'B-LOC',
  'score': 0.9997228,
  'index': 85,
  'word': 'Germany',
  'start': 411,
  'end': 418},
 {'entity': 'B-PER',
  's

In [ ]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

### Основные элементы пайплайна

Huggingface также позволяет получить доступ к своим пайплайнам на более низком уровне. Для вас есть две основные абстракции:
* `Tokenizer` - преобразует строки в идентификаторы токенов и обратно
* `Model` - модуль `nn.Module` в PyTorch с предварительно обученными весами

Вы можете использовать такие модели как часть вашего обычного кода в PyTorch: вставить их как слой в вашу модель, применить к пакету данных, провести обратное распространение ошибки, оптимизировать и т.д.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
model = transformers.AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
type(model)

transformers.models.bert.modeling_bert.BertModel

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# токенизировать батч входных данных. "pt" означает тензоры [p]y[t]orch
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt", max_length=512)

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [ ]:
tokens_info

{'input_ids': tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Теперь вы можете применить модель для получения эмбеддингов
with torch.no_grad():
    out = model(**tokens_info)

print(out['pooler_output'])

tensor([[-0.8854, -0.4722, -0.9392,  ..., -0.8081, -0.6955,  0.8748],
        [-0.9297, -0.5161, -0.9334,  ..., -0.9017, -0.7492,  0.9201]])


Transformers knowledge hub: https://huggingface.co/transformers/

In [ ]:
out.pooler_output.shape

torch.Size([2, 768])

In [ ]:
out.last_hidden_state

torch.Size([2, 15, 768])

## Анекдоты

In [ ]:
model(input_ids=tokens_info["input_ids"],token_type_ids=tokens_info["token_type_ids"] )

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="igorktech/rugpt3-joker-150k")

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Finetune

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

dataset = load_dataset('Romjiik/Russian_bank_reviews')
# dataset = load_dataset("csv", data_files="my_file.csv")

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("igorktech/rugpt3-joker-150k")
tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    return tokenizer(examples['review'], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Model
model = AutoModelForCausalLM.from_pretrained("igorktech/rugpt3-joker-150k")


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=0.2,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)

# Train the model
trainer.train()

# Save the model


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.json',
 './finetuned_model/merges.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [ ]:
from transformers import pipeline

model_path = "./finetuned_model"
text_generator = pipeline("text-generation", model=model_path)

In [ ]:
prompt = "JOKE:Заходит улитка в бар"
generated_text = text_generator(prompt, max_length=100, do_sample=True, temperature=0.7)
print(generated_text[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


JOKE:Заходит улитка в бар и говорит бармену - я хочу расплатиться картой Халва за покупки. Бармен - что за фигня!? У меня нет денег на покупку этой карты. Вы что себе позволяете? - говорит он - вы ж не хотите, чтобы эта карта была за вас платёжной системой, и не можете ею пользоваться. Вы не можете не понимать, что это за система, зачем она вам? В общем извините, я не понимаю. Если я
